##### The code here is taken from Copernicus Climate Change website to get netCDF datafile to retrieve Climate data from *Copernicus Climate Change Server*

##### Requirement (Windows with Anaconda VIrtual Environment)  
- Create an account here: https://cds.climate.copernicus.eu
- Install cdsapi via conda forge
- Install netCDF4 via conda forge 
- Go to the directory: C://Users//Username and create a file name .cdsapirc
- Copy the URL and Key form here and paste to the file created above: https://cds.climate.copernicus.eu/api-how-to
- Choose the data you need from here: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form

In [10]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
        'variable': '2m_temperature',
        'year': '2019',
        'month': '06',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
    },
    'download.nc')

2019-12-20 16:22:37,589 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2019-12-20 16:22:39,444 INFO Request is completed
2019-12-20 16:22:39,444 INFO Downloading http://136.156.133.41/cache-compute-0013/cache/data1/adaptor.mars.internal-1576838575.9693606-25162-3-e96a2d53-9fd5-45d4-95f1-3f9aa72fffc7.nc to download.nc (47.5M)
2019-12-20 16:23:45,324 INFO Download rate 738.9K/s                                                                    


Result(content_length=49845316,content_type=application/x-netcdf,location=http://136.156.133.41/cache-compute-0013/cache/data1/adaptor.mars.internal-1576838575.9693606-25162-3-e96a2d53-9fd5-45d4-95f1-3f9aa72fffc7.nc)

In [19]:
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2019-12-20 10:42:59 GMT by grib_to_netcdf-2.15.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -o /cache/data1/adaptor.mars.internal-1576838575.9693606-25162-3-e96a2d53-9fd5-45d4-95f1-3f9aa72fffc7.nc /cache/tmp/e96a2d53-9fd5-45d4-95f1-3f9aa72fffc7-adaptor.mars.internal-1576838575.9700694-25162-1-tmp.grib
    dimensions(sizes): longitude(1440), latitude(721), time(24)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 t2m(time,latitude,longitude)
    groups: 

In [24]:
import xarray as xr

ds = xr.open_dataset('download.nc')
df = ds.to_dataframe()

In [26]:
df.head()

t2m
latitude longitude time                           
90.0     0.0       2019-06-01 00:00:00  273.676971
                   2019-06-01 01:00:00  273.668121
                   2019-06-01 02:00:00  273.680511
                   2019-06-01 03:00:00  273.689362
                   2019-06-01 04:00:00  273.691132

In [27]:
#learn about the dataset here
#https://confluence.ecmwf.int/display/CKB/ERA5-Land+data+documentation#ERA5-Landdatadocumentation-Dataformat